In [ ]:
import time
import ast
import traceback
import random
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

import selenium
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from crawler import get_url, click_elm
from test import QuantusQA
from settings import filter_conditions

from database import AccessDataBase
db = AccessDataBase("quantus_test")

In [ ]:
import os, sys, time, shutil

variables = {}
def read_result(kind):
    for p in sys.path:
        if "Users" in p:
            user_name = p.split("/")[2]    
            download_dir = f"/Users/{user_name}/Downloads"
            bak_dir = os.path.join(download_dir, "bak")
            if not os.path.exists(bak_dir):
                os.mkdir(bak_dir)
            break

    files = os.listdir(download_dir)
    files = [os.path.join(download_dir, f) for f in files]
    files.sort(key=lambda x: os.path.getmtime(x))
    
    strategy_name = variables["strategy_name"]
    
    if kind == 0:
        # backtest
        # file = strategy_name + "_backtest_result.html"
        file = strategy_name + ".html"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()
        while not os.path.exists(path):
            time.sleep(10)
            # 백테스트에 100초 이상 소요되면 에러로 간주
            if time.time() - st > 100:
                ck = False
                print("backtest error")
                break

        if ck:
            
            _wd = get_url("file://" + path)
            soup = BeautifulSoup(_wd.page_source, "lxml")
            tbl = soup.find("div", {"id": "right"}).find("table")
            tbl_trs = tbl.find_all("tr")

            # Cumulative Return
            tbl_tr_td = tbl_trs[4].find_all("td")
            variables[tbl_tr_td[0].text] = tbl_tr_td[1].text
            # CAGR
            tbl_tr_td = tbl_trs[5].find_all("td")
            variables[tbl_tr_td[0].text] = tbl_tr_td[1].text
            # Max Drawdown	
            tbl_tr_td = tbl_trs[15].find_all("td")
            variables[tbl_tr_td[0].text] = tbl_tr_td[1].text
            
            shutil.move(path, bak)
            _wd.quit()
            
            
            
    elif kind == 1:
        # port extraction
        # file = strategy_name + "_portfolio_result.csv"
        file = "port_result.csv"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()            
        while not os.path.exists(path):
            time.sleep(10)
            # 백테스트에 100초 이상 소요되면 에러로 간주
            if time.time() - st > 60:
                print("port extraction error")
                ck = False
                break
        
        if ck:
            df = pd.read_csv(path)
            variables["portfolio"] = str(df["Name"].tolist())
            variables["industry"] = str(df["업종명"].tolist())
            shutil.move(path, bak)
            
            
    elif kind == 2:
        # 10분위 테스트
        # file = strategy_name + "_quantile_result.csv"
        file = "quantile_result.csv"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()            
        while not os.path.exists(path):
            time.sleep(10)
            # 10분위 테스트에 180초 이상 소요되면 에러로 간주
            if time.time() - st > 180:
                print("decile test error")
                ck = False
                break
        
        if ck:
            df = pd.read_csv(path)
            
            data = df.iloc[4, :].tolist()
            variables[data[0].strip() + "s"] = data[1:]

            data = df.iloc[5, :].tolist()
            variables[data[0].strip() + "s"] = data[1:]

            data = df.iloc[10, :].tolist()
            variables[data[0].strip() + "s"] = data[1:]
            shutil.move(path, bak)
            
    else:
        raise TypeError("올바른 kind(0: backtest, 1: port, 2: decile)를 입력하세요")


In [ ]:
variables["strategy_name"] = "켄피셔_대형주전략(미국)"

read_result(kind=0)
read_result(kind=1)
read_result(kind=2)

variables["version"] = "V1"

variables

In [ ]:
variables_list = [variables]
result_df = pd.DataFrame(variables_list)
result_df = result_df.astype("str")
result_df